In [16]:
import getpass
user_id = getpass.getuser()

import sys
sys.path.append(f"/home/{user_id}/OTTEHR/")
sys.path.append(f"/home/{user_id}/Statistical-Fairness-Test")

from collections import Counter
from common import *
from googletrans import Translator
import matplotlib.pyplot as plt
from mimic_common import *
from sklearn.decomposition import PCA

base_path = f"/home/{user_id}/OTTEHR/mimiciv_exp"
output_path = f"/home/{user_id}/OTTEHR/outputs/mimiciv"

The goal is to run Taskesen's statistical test to study the relationship between explanatory variable ICD code and label Insolin prescription based on protected attribute race (black and white)

Step 1: Select source and target samples

In [2]:
df = pd.read_csv(os.path.join(base_path, "mimiciv/final_admission.csv"), converters={'diagnosis_icd_code': literal_eval, 'procedure_icd_code': literal_eval, 'drug': literal_eval})
display(df)

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,...,edouttime,hospital_expire_flag,gender,anchor_age,anchor_year,anchor_year_group,dod,diagnosis_icd_code,procedure_icd_code,drug
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,...,2180-05-06 23:30:00,0,F,52,2180,2014 - 2016,2180-09-09,"[5723, 78959, 5715, 07070, 496, 29680, 30981, ...",[5491],"[Acetaminophen, Sodium Chloride 0.9% Flush, F..."
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,EW EMER.,P09Q6Y,EMERGENCY ROOM,HOME,Medicaid,...,2180-06-26 21:31:00,0,F,52,2180,2014 - 2016,2180-09-09,"[07071, 78959, 2875, 2761, 496, 5715, V08, 3051]",[5491],"[Rifaximin, Lactulose, Calcium Carbonate, Furo..."
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaN,EW EMER.,P60CC5,EMERGENCY ROOM,HOSPICE,Medicaid,...,2180-08-06 01:44:00,0,F,52,2180,2014 - 2016,2180-09-09,"[07054, 78959, V462, 5715, 2767, 2761, 496, V0...",[5491],"[Albumin 25% (12.5g / 50mL), Calcium Carbonate..."
3,10000117,27988844,2183-09-18 18:10:00,2183-09-21 16:30:00,NaN,OBSERVATION ADMIT,P75S70,WALK-IN/SELF REFERRAL,HOME HEALTH CARE,Other,...,2183-09-18 20:20:00,0,F,48,2174,2008 - 2010,NaN,"[S72012A, W010XXA, Y93K1, Y92480, K219, E7800,...",[0QS734Z],"[Senna, Bisacodyl, Cepacol (Sore Throat Lozeng..."
4,10000560,28979390,2189-10-15 10:30:00,2189-10-17 15:00:00,NaN,SURGICAL SAME DAY ADMISSION,P08ZLF,PHYSICIAN REFERRAL,HOME,Other,...,NaN,0,F,53,2189,2008 - 2010,NaN,"[1890, V1582, V1201]",[5551],"[Scopolamine Patch, Ketorolac, Lorazepam, Pota..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220848,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,NaN,EW EMER.,P75BG6,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,...,2149-01-08 18:12:00,0,F,46,2147,2017 - 2019,NaN,"[T8141XA, E1110, K632, D682, L02211, T83728A, ...","[0J980ZZ, 0WPF0JZ, 05HY33Z]","[Insulin, Potassium Chloride, Insulin, Insulin..."
220849,19999828,29734428,2147-07-18 16:23:00,2147-08-04 18:10:00,NaN,EW EMER.,P16C7J,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,...,2147-07-18 17:34:00,0,F,46,2147,2017 - 2019,NaN,"[T8131XA, T8183XA, K632, D6851, N390, Y838, Y9...","[0HR7X74, 0HBJXZZ, 0HBHXZZ, 02HV33Z, 3E0436Z]","[Insulin, Potassium Chloride (Powder), Acetami..."
220850,19999840,21033226,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,EW EMER.,P58A9J,EMERGENCY ROOM,DIED,Other,...,2164-09-10 14:46:00,1,M,58,2164,2008 - 2010,2164-09-17,"[3453, 51881, 5070, 5180, 42741, 43821, 43811,...","[9604, 9672, 966, 0331, 0331]","[Potassium Chloride, Bisacodyl, Phenytoin, 0.9..."
220851,19999840,26071774,2164-07-25 00:27:00,2164-07-28 12:15:00,NaN,EW EMER.,P506DE,EMERGENCY ROOM,HOME,Other,...,2164-07-25 01:20:00,0,M,58,2164,2008 - 2010,2164-09-17,"[43491, 43820, 34590, 43811, 4019, 2724, 3051]","[8891, 8841]","[Metoprolol Tartrate, Sodium Chloride 0.9% Fl..."


In [27]:
def select_drug_samples(df, group_name, drug, source, target, source_count = None, target_count = None):
    """ 
    Select rows in the dataframe df for source and target samples, return the results in \
        a format that is compatible to Taskesen et al's statistical test 
    
    :param df Dataframe: the dataframe to select samples from.
    :param str drug: drug name, used for preparing the labels array
    :param str source: source group name, e.g., White
    :param str target: target group name, e.g., Black
    :param int source_count: if specified, the number of samples for the source. \
        Otherwise, delete all rows that do not belong to target or source
    :param int target_count: if specified, the number of samples for the target. \
        Otherwise, delete all rows that do not belong to target or source
    """
    df_copy = copy.deepcopy(df)
    
    source_indices = []
    target_indices = []
    other_indices = []
    for index, row in df_copy.iterrows():
        if row[group_name] == source:
            source_indices.append(index)
        elif row[group_name] == target:
            target_indices.append(index)
        else:
            other_indices.append(index)

    
    # indices to delete from the dataframe
    if source_count is not None:
        delete_source_indices = random.sample(source_indices, len(source_indices)-source_count)
        delete_target_indices = random.sample(target_indices, len(target_indices)-target_count)

        other_indices.extend(delete_source_indices)
        other_indices.extend(delete_target_indices)

    df_copy = df_copy.drop(other_indices, axis=0, inplace=False)

    # prepare labels array, indicator of label attributes and \
    # sensitive_labels array: indicator of protected attributes
    labels = []
    sensitive_labels = []
    for _, row in df_copy.iterrows():
        if drug in row['drug']:
            labels.append(1)
        else:
            labels.append(0)

        if row[group_name] == source:
            sensitive_labels.append(1)
        elif row[group_name] == target:
            sensitive_labels.append(0)

    return df_copy, np.array(labels), np.array(sensitive_labels)

In [28]:
source_num = 10
target_num = 10
selected_df, labels, sensitive_labels = select_drug_samples(df, 'race', 'Insulin', 'WHITE', 'ASIAN', source_num, target_num)

In [29]:
display(selected_df)

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,...,edouttime,hospital_expire_flag,gender,anchor_age,anchor_year,anchor_year_group,dod,diagnosis_icd_code,procedure_icd_code,drug
10530,10499472,22928676,2181-04-07 20:32:00,2181-04-09 13:04:00,NaN,EW EMER.,P41V62,EMERGENCY ROOM,HOME,Other,...,2181-04-07 21:34:00,0,M,64,2181,2008 - 2010,NaN,"[41401, 4111, V4582, 4240, 4019, 25000, 53081,...","[0066, 3607, 0066, 3722, 0046, 0040, 8856, 8853]","[Insulin, Aspirin, Simvastatin, Atropine Sulfa..."
11098,10524716,21657089,2116-04-24 15:15:00,2116-05-14 15:23:00,NaN,EW EMER.,P233F6,EMERGENCY ROOM,REHAB,Other,...,2116-04-24 15:02:00,0,M,33,2116,2011 - 2013,NaN,"[82382, 8054, 8250, V1004, E8810, 3051, E0162,...","[7936, 7817, 7867, 7937, 7946]","[Ketorolac, Gabapentin, Cephalexin, CloniDINE,..."
25830,11198819,21545922,2161-01-03 07:15:00,2161-01-08 16:30:00,NaN,DIRECT EMER.,P38RSS,PHYSICIAN REFERRAL,HOME,Medicaid,...,NaN,0,F,64,2160,2011 - 2013,NaN,"[42731, 42789, 41401, 79092, 30000, 311, 71590...","[3734, 3726, 3727, 3726]","[Potassium Chloride, Potassium Chloride, Prome..."
33764,11554437,28139521,2121-08-12 12:15:00,2121-08-13 14:50:00,NaN,ELECTIVE,P01LRQ,PHYSICIAN REFERRAL,HOME,Other,...,NaN,0,F,52,2121,2011 - 2013,2121-10-19,"[1983, V1011, 73679, 496, 4019, 30000, 73390, ...",[0222],"[OxycoDONE (Immediate Release) , Dexamethasone..."
37311,11717234,27436911,2168-04-08 20:06:00,2168-04-12 18:00:00,NaN,EW EMER.,P74FFH,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,...,2168-04-08 22:10:00,0,M,88,2163,2008 - 2010,2168-06-26,"[0389, 486, 78552, 99592, 4241, V707, 41401, 4...",[3897],"[Furosemide, Sterile Water, Potassium Chloride..."
43828,12016339,24767398,2179-08-25 23:46:00,2179-09-06 19:09:00,NaN,EW EMER.,P94D9F,EMERGENCY ROOM,HOME,Medicaid,...,2179-08-26 01:25:00,0,M,65,2179,2014 - 2016,2179-12-15,"[1976, 56089, 1961, 1977, 1987, 5692, 1541, 53...",[4686],"[Rifaximin, Potassium Chloride, 0.9% Sodium Ch..."
73452,13332955,28402887,2152-04-24 07:15:00,2152-05-03 14:42:00,NaN,SURGICAL SAME DAY ADMISSION,P13NQ3,PHYSICIAN REFERRAL,HOME,Medicare,...,NaN,0,F,73,2152,2011 - 2013,2156-02-10,"[1589, 5184, 20410, 2762, 78959, 1978, 1975, 2...","[544, 4573, 6561, 6849, 415, 0390, 9915]","[Enoxaparin Sodium, 0.9% Sodium Chloride, Calc..."
84726,13855469,20029195,2138-10-25 10:51:00,2138-10-27 15:07:00,NaN,EW EMER.,P886Y0,EMERGENCY ROOM,HOME,Other,...,2138-10-25 12:00:00,0,M,73,2138,2011 - 2013,NaN,"[41011, 41401, 4142]","[0066, 3607, 0066, 0045, 0040, 9920, 3723, 8856]","[5% Dextrose, Heparin Sodium, Warfarin, Hepari..."
88680,14030918,25460412,2171-01-01 00:00:00,2171-01-03 11:40:00,NaN,SURGICAL SAME DAY ADMISSION,P39FGY,PHYSICIAN REFERRAL,HOME,Other,...,NaN,0,M,66,2169,2014 - 2016,NaN,"[Z472, E669, E785, G4733, M21371, Z8661, Z981,...","[0SP304Z, 0SP004Z, 0SU00KZ, 0SU30KZ]","[Lisinopril, Morphine Sulfate, Multivitamins, ..."
95733,14347468,26368410,2148-04-23 14:29:00,2148-04-28 13:48:00,NaN,ELECTIVE,P41R5N,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,...,NaN,0,M,54,2148,2008 - 2010,NaN,"[4240, 2762, 4019, 32723, 2720, 42731, 42789]","[3512, 3722, 3961, 8856]","[Bag, Magnesium Sulfate, Potassium Chloride, O..."


In [30]:
Counter(labels)

Counter({0: 14, 1: 6})

In [31]:
Counter(sensitive_labels)

Counter({0: 10, 1: 10})

In [32]:
def gen_feature(df, feature_code_name):
    """ 
    Generate source features and target features from dataframe df
    :param str feature_code_name: name of the input. For the experiments on ICD codes vs. duration, the input_name is 'ICD codes'

    """
    unique_code_dict, num_codes = find_unique_code(df, ICD_name = feature_code_name)

    # Prepare source
    features = np.empty(shape=[df.shape[0], num_codes])
    feature_index = 0
    for _, row in df.iterrows():
        code_ind = np.zeros(num_codes)
        for code in row[feature_code_name]:
            code_ind[unique_code_dict[code]] += 1
        features[feature_index] = code_ind
        feature_index += 1

    return features

In [33]:
features = gen_feature(selected_df, 'diagnosis_icd_code')

In [34]:
pca = PCA(n_components=10)
reps = pca.fit_transform(features)

In [35]:
print(features.shape, reps.shape)

(20, 165) (20, 10)


Step 2: Run Taskesen's statistical test

In [36]:
from load_data import *
from matplotlib import colors as mcolors
from random import seed
from functions import *
import time
from LogisticRegression import LogisticRegression
import datetime
from scipy.stats import gamma

In [37]:
clf = LogisticRegression(fit_intercept=False)
clf.fit(X=reps, y=labels)
beta = clf.coef_
print(beta)

[ 11.57821148 -38.59114675  35.4617686   24.24320191 -19.64411749
 -23.05514128  65.27668416 -64.70863124  -9.52334248 -43.71877059]


/home/wanxinli/miniconda3/lib/python3.9/site-packages/cvxpy/problems/problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


In [38]:
data_tuple = [reps, np.array(sensitive_labels), np.array(labels)]
range_gamma = [-5, 5]
range_of_k = np.linspace(0, 1/8, 100)
tol = 1e-10

In [40]:
test_marginals = np.reshape(get_marginals(sensitives=sensitive_labels, target=labels), [2, 2])
c = limiting_dist_EQOPP(X=reps, a=sensitive_labels, y=labels, beta=beta, marginals=test_marginals)
theta = 2 * c
print(c, theta)

8.575058959481149e-05 0.00017150117918962298


In [41]:
dist, optimum_ks, gamma_opt = calculate_distance_prob_eqopp_with_opt_params(data_tuple=data_tuple,
                                          function_of_gamma=f_gamma,
                                          range_gamma=range_gamma,
                                          k_opt_fcn=k_opt,
                                          range_of_k=range_of_k,
                                          beta=beta, tol=tol)

/home/wanxinli/Statistical-Fairness-Test/functions.py:49: RuntimeWarning: overflow encountered in exp
  gamma * lambd / (1 + np.exp(gamma * lambd * LA.norm(beta, ord=2) ** 2 * k - beta.T @ x_hat)))  # objective function
/home/wanxinli/Statistical-Fairness-Test/functions.py:49: RuntimeWarning: overflow encountered in exp
  gamma * lambd / (1 + np.exp(gamma * lambd * LA.norm(beta, ord=2) ** 2 * k - beta.T @ x_hat)))  # objective function
/home/wanxinli/Statistical-Fairness-Test/functions.py:49: RuntimeWarning: overflow encountered in exp
  gamma * lambd / (1 + np.exp(gamma * lambd * LA.norm(beta, ord=2) ** 2 * k - beta.T @ x_hat)))  # objective function
/home/wanxinli/Statistical-Fairness-Test/functions.py:49: RuntimeWarning: overflow encountered in exp
  gamma * lambd / (1 + np.exp(gamma * lambd * LA.norm(beta, ord=2) ** 2 * k - beta.T @ x_hat)))  # objective function
/home/wanxinli/Statistical-Fairness-Test/functions.py:49: RuntimeWarning: overflow encountered in exp
  gamma * lambd / 

In [43]:
print(dist)

7.748117702988111e-11
